# Neresbot 0.1

Modelos:
* Convolução
* Long Short-Term Memory
* SARIMA

In [16]:
import pandas as pd
import requests
import json
from pprint import pprint
import yahooquery as yq
import os

In [17]:
download = True
indexing = False

# Download dos Dados

Os símbolos dos tickers são obtidos manualmente do site: https://www.dadosdemercado.com.br/bolsa/acoes

In [18]:
tickers = pd.read_csv("acoes-listadas.csv")

In [19]:
tickers

,Código,Nome
0,PETR4,Petrobras
1,MGLU3,Magazine Luiza
2,HAPV3,Hapvida
3,CIEL3,Cielo
4,B3SA3,B3
...,...,...
522,TKNO3,Tekno
523,COCE6,Coelce
524,MGEL3,Mangels
525,CTSA8,Santanense


https://yahooquery.dpguthrie.com/

É criado um dicionário que conterá, como chave, o símbolo do ticker e, como valor, o dataframe relacionando medidas e períodos (dias).

Em cada dataframe é adicionado o devido dia da semana, numericamente.

In [20]:
# Os símbolos das empresas brasileiras recebem o sufixo ".SA".

símbolos = tickers['Código'] + '.SA'

In [21]:
# Verificando símbolos inválidos.

print(yq.Ticker(símbolos, backoff_factor=1, progress=True, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True).invalid_symbols)

None


Ajuste dos dados:
* Download e armazenamento:
  * Os índices são resetados
  * É verificado se os dados possuem a coluna 'date':
    * É criada a coluna equivalente aos dias semanais
    * Armazenamento no dict dados1
    * Os dados são salvos na pasta 'dados'

In [22]:
test = pd.DataFrame({'a':[1,2,3,4,5]})

In [23]:
test['tester'] = 0 if 'tester' not in test.columns else test['tester']

In [24]:
test

,a,tester
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [30]:
# 2
# Download dos dados e alocação dos dataframes no dicionário "dados1"

if download:
    dados1 = {}
    falhas = []

    for ticker in símbolos:
        query = yq.Ticker(ticker, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period='1y', interval='1d',adj_timezone=True)
        except:
            falhas.append(ticker)
            print(ticker,"falhou.")
        else:
            histórico = histórico.reset_index()
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dados1
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date') # Organiza por data
                
                dados1[ticker] = histórico # Salva tudo em dados1
            else:
                print(ticker,"não contém datas, mas contém:")
                print(histórico.columns)

    print(len(dados1),"tickers salvos com sucesso.")
    print(len(falhas),"tickers apresentaram falha")

    Medidas = []
    for tckr in dados1:
        for col in dados1[tckr].columns:
            if col not in Medidas:
                Medidas.append(col)
                print(col)
    for tckr in dados1:
        for col in Medidas:
            if col not in dados1[tckr].columns:
                dados1[tckr][col] = 0

    # Verifica se a pasta de destino existe, se não, cria-a
    if not os.path.exists('dados'):
        os.makedirs('dados')
    # Itera pelo dicionário e salva cada dataframe como um arquivo CSV
    for tckr, df in dados1.items():
        caminho_arquivo = os.path.join('dados', f'{tckr}.csv')  # Caminho completo para o arquivo CSV
        df.to_csv(caminho_arquivo, index=True)  # Salva o dataframe como arquivo CSV
        
        print(f'Arquivo {tckr}.csv salvo com sucesso.')

WIZS3.SA falhou.
LLIS3.SA falhou.
BRML3.SA falhou.
DMMO3.SA falhou.
GETT3.SA falhou.
GETT4.SA falhou.
SULA4.SA falhou.
SULA3.SA falhou.
CEPE5.SA falhou.
CEPE6.SA falhou.
BKBR3.SA falhou.
MTIG4.SA falhou.
BLUT4.SA falhou.
BLUT3.SA falhou.
MODL4.SA falhou.
CARD3.SA falhou.
FNCN3.SA falhou.
LCAM3.SA falhou.
BIDI4.SA falhou.
BIDI3.SA falhou.
EEEL4.SA falhou.
EEEL3.SA falhou.
BBRK3.SA falhou.
CESP6.SA falhou.
CESP3.SA falhou.
CESP5.SA falhou.
MOSI3.SA falhou.
POWE3.SA falhou.
GNDI3.SA falhou.
LAME4.SA falhou.
LAME3.SA falhou.
OMGE3.SA falhou.
IGTA3.SA falhou.
JPSA3.SA falhou.
BRDT3.SA falhou.
JBDU4.SA falhou.
JBDU3.SA falhou.
HGTX3.SA falhou.
CCPR3.SA falhou.
DTEX3.SA falhou.
VVAR3.SA falhou.
PNVL4.SA falhou.
TESA3.SA falhou.
BTOW3.SA falhou.
LINX3.SA falhou.
BTTL3.SA falhou.
GPCP3.SA falhou.
GPCP4.SA falhou.
SMLS3.SA falhou.
MMXM3.SA falhou.
BSEV3.SA falhou.
CNTO3.SA falhou.
TIET4.SA falhou.
TIET3.SA falhou.
CEPE3.SA falhou.
CALI4.SA falhou.
BPAR3.SA falhou.
APTI4.SA falhou.
CTSA8.SA não c

In [33]:
if not download:
    dados1 = {}  # Dicionário vazio para armazenar as tabelas

    # Percorre todos os arquivos na pasta
    for nome_arquivo in os.listdir('dados'):
        if nome_arquivo.endswith('.csv'):  # Verifica se o arquivo é um arquivo CSV
            caminho_arquivo = os.path.join('dados', nome_arquivo)  # Caminho completo para o arquivo

            histórico = pd.read_csv(caminho_arquivo)
            histórico['date'] = pd.to_datetime(histórico['date'])
            histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
            histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
            histórico['date'] = histórico['date'].dt.date # Remove horário
            histórico = histórico.sort_values('date') # Organiza por data
            dados1[os.path.splitext(nome_arquivo)[0]] = histórico

    # Imprime o dicionário com as tabelas
    print(len(dados1))

    print(dados1['PETR4.SA'])

    for i in dados1:
        print(i)

466
     Unnamed: 0    symbol        date       open       high        low  \
0             0  PETR4.SA  2022-06-19  26.020000  28.250000  25.910000   
1             1  PETR4.SA  2022-06-20  27.799999  28.160000  26.690001   
2             2  PETR4.SA  2022-06-21  26.580000  27.549999  26.389999   
3             3  PETR4.SA  2022-06-22  26.980000  27.379999  26.120001   
4             4  PETR4.SA  2022-06-23  26.760000  27.090000  26.180000   
..          ...       ...         ...        ...        ...        ...   
245         245  PETR4.SA  2023-06-12  29.240000  29.240000  28.590000   
246         246  PETR4.SA  2023-06-13  28.940001  30.100000  28.879999   
247         247  PETR4.SA  2023-06-14  30.200001  30.850000  29.280001   
248         248  PETR4.SA  2023-06-15  29.230000  29.920000  29.040001   
249         249  PETR4.SA  2023-06-19  29.629999  30.459999  29.549999   

         close     volume   adjclose  dividends  weekday  splits  
0    27.620001   92581800  16.437548   0

In [31]:
# Verificando se todas as colunas 'date' são iguais 

ticker_divergentes = pd.DataFrame(columns=dados1.keys(), index=dados1.keys())

for column in dados1.keys():
    for index in dados1.keys():
        if dados1[column]['date'].equals(dados1[index]['date']):
            ticker_divergentes.at[index,column] = True
        else:
            ticker_divergentes.at[index,column] = False

ticker_divergentes.to_csv('ticker_divergentes.csv')
ticker_divergentes

,PETR4.SA,MGLU3.SA,HAPV3.SA,CIEL3.SA,B3SA3.SA,VIIA3.SA,ITSA4.SA,CVCB3.SA,VALE3.SA,BBDC4.SA,...,CASN4.SA,EMAE3.SA,VSPT3.SA,MTIG3.SA,FIGE4.SA,LUXM3.SA,TKNO3.SA,COCE6.SA,MGEL3.SA,MMAQ4.SA
PETR4.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
MGLU3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
HAPV3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
CIEL3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
B3SA3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LUXM3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
TKNO3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True
COCE6.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True
MGEL3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True


# Convolução

## Rótulos

Cada linha analisa se há aumento expressivo nos próximos 5 períodos.

Se não houver, o rótulo será 0; Caso contrário, será o número de dias em que o aumento ocorre.  
Há uma variável ajustável para determinar o nível do aumento.

In [185]:
# Coluna para os rótulos

for i,j in dados1.items():
    j['rótulos'] = 0

In [186]:
# Metaparâmetros

Aumento = 0.2

Prospecção = 5

In [187]:
for key, df in dados1.items():                  # Para cada ticker, tabela:
    df = df.reset_index()                         # reseta o índice da tabela
    for i, row in df.iterrows():                  # para cada índice, coluna da tabela:
        price_now = row['close']                     # o preço atualé o número contido na coluna 'close'

        price_contester = price_now
        for j in range(i+1, i+1+Prospecção):         # para cada número j entre o índice atual e o índice + prospecção (+1 pois o último desconsidera)
            if j < len(df):                             # se j é menor que o tamanho da tabela:
                post_high = df.loc[j, 'high']              # o preço avaliado é o o número no índice j, na coluna 'high'
                if (post_high >= (1 + Aumento)*price_now) and (post_high > price_contester):   # se o preço avaliado é maior ou igual ao aumento esperado:
                    df.loc[i, 'rótulos'] = j - i              # o valor em 'rótulo' é o índice em j menos o índice em i (que é os períodos à frente)
                    price_contester = post_high
                    if (j-i) > 0:
                        print(row['symbol'],row['date'],(j-i),(((post_high/price_now)-1)*100))
    
    # print(df.head()) #??

MGLU3.SA 2022-07-01 00:00:00 4 20.45454717864672
MGLU3.SA 2022-07-04 00:00:00 2 22.535199758939804
MGLU3.SA 2022-07-04 00:00:00 3 24.41314333276472
MGLU3.SA 2022-07-04 00:00:00 5 26.291075713229706
MGLU3.SA 2022-07-05 00:00:00 5 27.310917045965887
MGLU3.SA 2022-07-06 00:00:00 4 21.199998855590827
MGLU3.SA 2022-07-07 00:00:00 5 20.312499708961695
MGLU3.SA 2022-07-28 00:00:00 5 24.264708782665423
MGLU3.SA 2022-07-29 00:00:00 4 31.00776024774834
MGLU3.SA 2022-08-01 00:00:00 3 24.264708782665423
MGLU3.SA 2022-08-01 00:00:00 5 27.94117582607436
MGLU3.SA 2022-08-02 00:00:00 2 24.723249699733362
MGLU3.SA 2022-08-02 00:00:00 4 28.413283029066648
MGLU3.SA 2022-08-08 00:00:00 5 23.853217564464522
MGLU3.SA 2022-08-09 00:00:00 4 31.49351594339138
MGLU3.SA 2022-08-09 00:00:00 5 39.28571649738948
MGLU3.SA 2022-08-10 00:00:00 3 23.100311176106537
MGLU3.SA 2022-08-10 00:00:00 4 30.395137130541784
MGLU3.SA 2022-08-11 00:00:00 2 33.22369215643648
MGLU3.SA 2022-08-11 00:00:00 3 41.11842156859977
MGLU3.SA

In [177]:
for tckr in dados1:
    print(dados[tckr].tail(4))

              symbol   open       high        low      close  volume  \
date                                                                   
2023-06-13  AALR3.SA  23.33  23.469999  23.209999  23.299999  272600   
2023-06-14  AALR3.SA  23.25  23.450001  23.160000  23.350000  154500   
2023-06-15  AALR3.SA  23.27  23.309999  23.129999  23.219999  212600   
2023-06-16  AALR3.SA  23.23  23.250000  23.110001  23.240000   99700   

             adjclose  weekday  rótulos  
date                                     
2023-06-13  23.299999        1        0  
2023-06-14  23.350000        2        0  
2023-06-15  23.219999        3        0  
2023-06-16  23.240000        4        0  
              symbol   open       high        low      close   volume  \
date                                                                    
2023-06-13  ABCB4.SA  18.35  18.360001  17.809999  17.850000   779100   
2023-06-14  ABCB4.SA  18.08  18.120001  17.600000  17.889999  1306600   
2023-06-15  ABCB4.SA  1

## Incoerência com as datas de negociação

Em análise, nota-se que há divergência entre as datas em que cada ativo é negociado. Em grande parte há algum padrão, mas não se sabe quantos.

## Transformação das matrizes: Separando os dias como objetos numpy.

Classes que contenham as matrizes, os rótulos e a identificação do dia.

In [179]:
# Todos os dataframes:

df_concatenado = pd.concat(dados1.values(), axis=1)

In [183]:
df_concatenado.to_csv('df_concatenado.csv',index=True)

O último dia não está incluso no treinamento por não possuir rótulo.

Dimensões:
* Medidas:
  * Dia da semana
  * Dia do ano
  * Abertura
  * Mínimo
  * Máximo
  * Fechamento
  * Fechamento Ajustado
  * Volume
* tickers
* Dias (atual e anteriores)

Há uma variável ajustável para determinar quantos dias serão olhados para trás.

Um dicionário de rótulos, um rótulo para cada ticker, é atrelado a cada matriz.

## Separação entre Treino, Teste e Validação.

É preciso garantir que as matrizes sejam embaralhadas, mas que os rótulos permaneçam atrelados adequadamente.

## Modelo

* Para cada ticker (a partir do dicionário de rótulos):  
    * O modelo é treinado seguindo aquele ticker como rótulo (com uma função)  
    * O modelo é validado  
    * Se o modelo é válido:  
        * A previsão é tentada:  
        * Se o resultado é diferente de zero E a probabilidade é alta:  
                * O deploy é salvo